# NLP Pipeline 

In [35]:
from pymongo import MongoClient
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer

## Connect to the Mongo clinical_trials database 

In [2]:
def connect_to_mongo(database, collection):
    
    """
    Opens a connection to a specified Mongo DB location
    
    Input Parameters:
    database: name of database to connect to or create (str)
    collection: name of collection to connect to or create (str)
    
    Returns:
    The connection object for the database without a collection specified
    The connection object for a specific Mongo location (database & collection)
    """
    
    client = MongoClient()
    db = client[database]
    mongo_loc = db[collection]
    return db, mongo_loc

In [3]:
trials_loc, eligibility_loc = connect_to_mongo('clinical_trials', 'eligibilities')

### Create a Mongo cursor 

In [61]:
doc_cursor = eligibility_loc.find()

In [62]:
test_docs = doc_cursor[0:1]

In [63]:
print(test_docs[0])

{'_id': ObjectId('5b734462aded604ea198f141'), 'study_id': 'NCT00864942', 'minimum_age': '18 Years', 'maximum_age': 'N/A', 'gender': 'All', 'inclusion_criteria': ['Documented relapsed or refractory B-cell NHL; CD-20 positive tumor. Indolent NHL: follicular B-cell lymphoma, diffuse small lymphocytic lymphoma, lymphoplasmacytic lymphoma, marginal zone lymphoma, transformed aggressive lymphomas, mantle cell lymphoma and chronic lymphocytic leukemia', 'Maximum of 6 prior chemotherapy regimens. Prior rituximab is allowed.', 'Bidimensionally measurable disease', 'ECOG performance status 0-2', 'Absolute neutrophil count >/= 1000 and platelet count >/= 50,000', 'Serum creatinine </= 1.5 mg/dL', 'Adequate hepatic function', 'Estimated life expectancy of at least 3 months', 'All study participants must be registered into the mandatory RevAssist program and be willing and able to comply with the requirements of RevAssist', 'Able to take aspirin 81 mg daily as prophylactic anticoagulation'], 'exclu

## Clean the text 

1. get the doc
2. decide how to segment the eligibility data - first pass, try to do all inclusion data for a study as one entry, all exclusion data for a study as an entry. This might require concatenating all the elements in the inclusion_criteria into one long element:  
`" ".join(my_list)`
3. separately clean inclusion_criteria and exclusion_criteria
4. add the cleaned_inclusion_criteria and cleaned_exclusion_criteria as part of that doc record - find it by the study_id

In [64]:
for doc in test_docs: 
    inclusion_text = " ".join(doc['inclusion_criteria'])
    exclusion_text = " ".join(doc['exclusion_criteria'])
    no_punc_inclusion = inclusion_text.translate(str.maketrans('','',string.punctuation))
    # need to remove double white spaces
    # need to remove stand-alone numbers (will keep in genetic markers like cd20)
    # just focus on inclusion_text for now
    # remove stop words
    inclusion_tokens = no_punc_inclusion.split(' ')

In [65]:
inclusion_tokens

['Documented',
 'relapsed',
 'or',
 'refractory',
 'Bcell',
 'NHL',
 'CD20',
 'positive',
 'tumor',
 'Indolent',
 'NHL',
 'follicular',
 'Bcell',
 'lymphoma',
 'diffuse',
 'small',
 'lymphocytic',
 'lymphoma',
 'lymphoplasmacytic',
 'lymphoma',
 'marginal',
 'zone',
 'lymphoma',
 'transformed',
 'aggressive',
 'lymphomas',
 'mantle',
 'cell',
 'lymphoma',
 'and',
 'chronic',
 'lymphocytic',
 'leukemia',
 'Maximum',
 'of',
 '6',
 'prior',
 'chemotherapy',
 'regimens',
 'Prior',
 'rituximab',
 'is',
 'allowed',
 'Bidimensionally',
 'measurable',
 'disease',
 'ECOG',
 'performance',
 'status',
 '02',
 'Absolute',
 'neutrophil',
 'count',
 '',
 '1000',
 'and',
 'platelet',
 'count',
 '',
 '50000',
 'Serum',
 'creatinine',
 '',
 '15',
 'mgdL',
 'Adequate',
 'hepatic',
 'function',
 'Estimated',
 'life',
 'expectancy',
 'of',
 'at',
 'least',
 '3',
 'months',
 'All',
 'study',
 'participants',
 'must',
 'be',
 'registered',
 'into',
 'the',
 'mandatory',
 'RevAssist',
 'program',
 'and',
 'b

In [68]:
cleaned_inclusion = []
stemmer = PorterStemmer()
for token in inclusion_tokens: 
    low_word = token.lower()
    low_stem = stemmer.stem(low_word)
    cleaned_inclusion.append(low_stem)

In [69]:
cleaned_inclusion

['document',
 'relaps',
 'or',
 'refractori',
 'bcell',
 'nhl',
 'cd20',
 'posit',
 'tumor',
 'indol',
 'nhl',
 'follicular',
 'bcell',
 'lymphoma',
 'diffus',
 'small',
 'lymphocyt',
 'lymphoma',
 'lymphoplasmacyt',
 'lymphoma',
 'margin',
 'zone',
 'lymphoma',
 'transform',
 'aggress',
 'lymphoma',
 'mantl',
 'cell',
 'lymphoma',
 'and',
 'chronic',
 'lymphocyt',
 'leukemia',
 'maximum',
 'of',
 '6',
 'prior',
 'chemotherapi',
 'regimen',
 'prior',
 'rituximab',
 'is',
 'allow',
 'bidimension',
 'measur',
 'diseas',
 'ecog',
 'perform',
 'statu',
 '02',
 'absolut',
 'neutrophil',
 'count',
 '',
 '1000',
 'and',
 'platelet',
 'count',
 '',
 '50000',
 'serum',
 'creatinin',
 '',
 '15',
 'mgdl',
 'adequ',
 'hepat',
 'function',
 'estim',
 'life',
 'expect',
 'of',
 'at',
 'least',
 '3',
 'month',
 'all',
 'studi',
 'particip',
 'must',
 'be',
 'regist',
 'into',
 'the',
 'mandatori',
 'revassist',
 'program',
 'and',
 'be',
 'will',
 'and',
 'abl',
 'to',
 'compli',
 'with',
 'the',
 'r

In [50]:
test_token = inclusion_tokens[5]

In [16]:
doc_cursor.close()

# Next steps 

## Improving the pipeline 

* Refactor `connect_to_mongo()` function so we don't have to reconnect when switching databases (might be able to pull this into other functions later on)

## Error checking and production aspects to add   

* Error messages
* docstrings
* create functions
* comments to explain hard-coding choices or why an approach was used